pascalvoc: resize original images

Strategy:
1. Read: Images RGB / Masks RGB (original dataset)
2. Pad
3. Save: Images RGB / Masks RGB

In [ ]:
padsize = (640,640)

environment

In [ ]:
using Pkg
envpath = expanduser("~/envs/d11/")
Pkg.activate(envpath)

In [ ]:
# libs
using Images
using DataFrames
using CSV
using FLoops
using MLUtils: splitobs, randobs
using Random

In [ ]:
# private libs
using PreprocessingImages; p=PreprocessingImages

In [ ]:
# output folders
workpath = pwd() * "/"
workpath = replace(workpath, homedir() => "~")

resizedTrainsetPath = workpath * "images/resized/train/"
resizedValidsetPath = workpath * "images/resized/valid/"
resizedTestsetPath  = workpath * "images/resized/test/"
mkpath(expanduser(resizedTrainsetPath))
mkpath(expanduser(resizedValidsetPath))
mkpath(expanduser(resizedTestsetPath))

input dataset

In [ ]:
fpfn = expanduser(workpath) * "dftrain-coi-balanced.csv"
dftrain_coi_balanced = CSV.read(fpfn, DataFrame)

fpfn = expanduser(workpath) * "dftrain-cow-balanced.csv"
dftrain_cow_balanced = CSV.read(fpfn, DataFrame)

fpfn = expanduser(workpath) * "dfvalid-coi-balanced.csv"
dfvalid_coi_balanced = CSV.read(fpfn, DataFrame)

fpfn = expanduser(workpath) * "dfvalid-cow-balanced.csv"
dfvalid_cow_balanced = CSV.read(fpfn, DataFrame)

fpfn = expanduser(workpath) * "dftest-classes.csv"
dftest_classes = CSV.read(fpfn, DataFrame)

size(dftrain_coi_balanced), size(dftrain_cow_balanced), size(dfvalid_coi_balanced), size(dfvalid_cow_balanced), size(dftest_classes)

In [ ]:
first(dftrain_coi_balanced,3)

In [ ]:
# create datasets with resized images
dftrain_coi_resized = copy(dftrain_coi_balanced)
dfvalid_coi_resized = copy(dfvalid_coi_balanced)

dftrain_cow_resized = copy(dftrain_cow_balanced)
dfvalid_cow_resized = copy(dfvalid_cow_balanced)

dftest_classes_resized = copy(dftest_classes);

In [ ]:
@. dftrain_coi_resized.X = resizedTrainsetPath * basename(dftrain_coi_resized.X)
@. dfvalid_coi_resized.X = resizedValidsetPath * basename(dfvalid_coi_resized.X)

@. dftrain_cow_resized.X = resizedTrainsetPath * basename(dftrain_cow_resized.X)
@. dfvalid_cow_resized.X = resizedValidsetPath * basename(dfvalid_cow_resized.X)

@. dftest_classes_resized.X = resizedTestsetPath * basename(dftest_classes_resized.X);

In [ ]:
@. dftrain_coi_resized.y = resizedTrainsetPath * basename(dftrain_coi_resized.y)
@. dfvalid_coi_resized.y = resizedValidsetPath * basename(dfvalid_coi_resized.y)

@. dftrain_cow_resized.y = resizedTrainsetPath * basename(dftrain_cow_resized.y)
@. dfvalid_cow_resized.y = resizedValidsetPath * basename(dfvalid_cow_resized.y)

@. dftest_classes_resized.y = resizedTestsetPath * basename(dftest_classes_resized.y);

In [ ]:
first(dftrain_coi_resized,3)

resize

In [ ]:
# # debug
# dftrain_coi_balanced = dftrain_coi_balanced[1:10, :]
# dfvalid_coi_balanced = dfvalid_coi_balanced[1:10, :]
# dftest_classes = dftest_classes[1:10, :]

# dftrain_coi_resized = dftrain_coi_resized[1:10, :]
# dfvalid_coi_resized = dfvalid_coi_resized[1:10, :]
# dftest_classes_resized = dftest_classes_resized[1:10, :];

In [ ]:
### only coi images need to be resized
dfs_in  = [dftrain_coi_balanced, dfvalid_coi_balanced, dftest_classes]
dfs_out = [dftrain_coi_resized, dfvalid_coi_resized, dftest_classes_resized];

# resize
for (df_in, df_out) in zip(dfs_in, dfs_out)
    N = size(df_in, 1)

    # images
    @floop for i in 1:N
        # resize all images
        local fpfn_in = expanduser(df_in.X[i])
        img = Images.load(fpfn_in)                       # read
        img = p.center_padview(img, padsize) .|> RGB     # pad
        local fpfn_out = expanduser(df_out.X[i])
        Images.save(fpfn_out, img)                       # save
    end

    # masks
    @floop for i in 1:N
        if !df_in.coi[i]   continue   end                # only coi masks need to be resized
        local fpfn_in = expanduser(df_in.y[i])
        img = Images.load(fpfn_in)                       # read
        img = p.center_padview(img, padsize) .|> Gray    # pad
        local fpfn_out = expanduser(df_out.y[i])
        Images.save(fpfn_out, img)                       # save
    end
end

In [ ]:
size(dftrain_coi_resized), size(dfvalid_coi_resized), size(dftest_classes_resized)

In [ ]:
# save dataframes
fpfn = expanduser(workpath) * "dftrain-coi-resized.csv"
CSV.write(fpfn, dftrain_coi_resized)
fpfn = expanduser(workpath) * "dfvalid-coi-resized.csv"
CSV.write(fpfn, dfvalid_coi_resized)

fpfn = expanduser(workpath) * "dftrain-cow-resized.csv"
CSV.write(fpfn, dftrain_cow_resized)
fpfn = expanduser(workpath) * "dfvalid-cow-resized.csv"
CSV.write(fpfn, dfvalid_cow_resized)

fpfn = expanduser(workpath) * "dftest-classes-resized.csv"
CSV.write(fpfn, dftest_classes_resized)